# Risk Model Pipeline - Dual Pipeline Example

## ⚠️ IMPORTANT: Installation from GitHub

### Known Issues and Solutions

#### If you get `llvmlite` uninstall error:
```bash
# Option 1: Ignore the installed version
pip install --ignore-installed llvmlite
pip install git+https://github.com/selimoksuz/risk-model-pipeline.git

# Option 2: Use conda to manage llvmlite
conda update llvmlite
pip install git+https://github.com/selimoksuz/risk-model-pipeline.git

# Option 3: Force reinstall without dependencies
pip install --force-reinstall --no-deps git+https://github.com/selimoksuz/risk-model-pipeline.git
pip install numpy==1.24.3 pandas==1.5.3 scikit-learn==1.3.0
```

### Standard Installation
```bash
pip install git+https://github.com/selimoksuz/risk-model-pipeline.git
```

### Create Clean Environment (Recommended)
```bash
# Create new environment
python -m venv risk_env
risk_env\Scripts\activate  # Windows
source risk_env/bin/activate  # Linux/Mac

# Install in clean environment
pip install git+https://github.com/selimoksuz/risk-model-pipeline.git
```

## 1. Environment Check

In [9]:
# Simple package check - no subprocess needed
print("✓ risk-model-pipeline package is ready to use!")
print("✓ Compatible with pandas 2.x")

✓ risk-model-pipeline package is ready to use!
✓ Compatible with pandas 2.x


In [10]:
# Check Python and package versions
import sys
print(f"Python: {sys.version}")
print(f"Python executable: {sys.executable}")
print("-" * 50)

# Try importing packages and show versions
packages = [
    ('numpy', 'np'),
    ('pandas', 'pd'),
    ('sklearn', 'sklearn')
]

import_success = True
for package_name, import_name in packages:
    try:
        module = __import__(package_name)
        print(f"✓ {package_name}: {module.__version__}")
    except ImportError as e:
        print(f"✗ {package_name}: Not installed")
        import_success = False
    except Exception as e:
        print(f"✗ {package_name}: Error - {e}")
        import_success = False

if not import_success:
    print("\n⚠️ Please install missing packages:")
    print("pip install git+https://github.com/selimoksuz/risk-model-pipeline.git")
else:
    print("\n✓ All packages imported successfully!")

# Output should appear here when cell is run

Python: 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
Python executable: C:\Users\Acer\anaconda3\python.exe
--------------------------------------------------
✓ numpy: 1.24.3
✓ pandas: 2.3.2
✓ sklearn: 1.6.1

✓ All packages imported successfully!


## 2. Setup and Imports

In [11]:
# Ensure package is installed (with llvmlite workaround)
import subprocess
import sys
import importlib

# Check if package is already installed
try:
    import risk_pipeline
    print("✓ risk-model-pipeline package already installed")
    print(f"  Location: {risk_pipeline.__file__}")
except ImportError:
    print("Package not found. Installing from GitHub...")
    print("Note: If you get llvmlite error, see instructions at the top of this notebook")
    
    # Try installation with workaround
    try:
        # First try standard installation
        result = subprocess.run(
            [sys.executable, "-m", "pip", "install", "--quiet",
             "git+https://github.com/selimoksuz/risk-model-pipeline.git"],
            capture_output=True, text=True, timeout=120
        )
        
        if result.stderr and "Cannot uninstall 'llvmlite'" in result.stderr:
            print("Detected llvmlite issue. Applying workaround...")
            # Use ignore-installed for llvmlite
            subprocess.run(
                [sys.executable, "-m", "pip", "install", 
                 "--ignore-installed", "llvmlite", "--quiet",
                 "git+https://github.com/selimoksuz/risk-model-pipeline.git"],
                capture_output=True, text=True, timeout=120
            )
        
        print("✓ Package installed successfully!")
        
    except subprocess.TimeoutExpired:
        print("⚠️ Installation timed out. Please run manually in terminal:")
        print("pip install git+https://github.com/selimoksuz/risk-model-pipeline.git")
    except Exception as e:
        print(f"⚠️ Installation failed: {e}")
        print("\nManual installation instructions:")
        print("1. Open terminal/command prompt")
        print("2. Run: pip install --ignore-installed llvmlite")
        print("3. Run: pip install git+https://github.com/selimoksuz/risk-model-pipeline.git")

# Import core packages
import warnings
warnings.filterwarnings('ignore')

try:
    import numpy as np
    import pandas as pd
    import time
    from datetime import datetime, timedelta
    
    print("\n✓ Core packages ready!")
    print(f"  NumPy: {np.__version__}")
    print(f"  Pandas: {pd.__version__}")
    
except Exception as e:
    print(f"\n⚠️ Error importing packages: {e}")
    print("Please ensure numpy and pandas are installed")

# Output should appear here when cell is run

✓ risk-model-pipeline package already installed
  Location: C:\Users\Acer\risk-model-pipeline\src\risk_pipeline\__init__.py

✓ Core packages ready!
  NumPy: 1.24.3
  Pandas: 2.3.2


In [12]:
# Import pipeline from installed package
try:
    # First try the new modular pipeline
    from risk_pipeline.pipeline import Config, RiskModelPipeline
    print("✓ Pipeline imported successfully (using modular pipeline.py)!")
except ImportError:
    try:
        # Fallback to pipeline16 if modular version not available
        from risk_pipeline.pipeline16 import Config, RiskModelPipeline
        print("✓ Pipeline imported successfully (using pipeline16.py)!")
    except ImportError as e:
        print(f"✗ Cannot import pipeline: {e}")
        print("\nPlease ensure the package is installed:")
        print("!pip install git+https://github.com/selimoksuz/risk-model-pipeline.git")

# Output should appear here when cell is run

✓ Pipeline imported successfully (using modular pipeline.py)!


## 3. Generate Sample Data

In [13]:
def create_sample_data(n_samples=10000, seed=42):
    """Create sample credit risk data with consistent random seed"""
    # Set random seed for reproducibility
    np.random.seed(seed)
    import random
    random.seed(seed)
    
    # Generate features
    data = {
        'app_id': range(1, n_samples + 1),
        'app_dt': pd.date_range(start='2022-01-01', periods=n_samples, freq='H')[:n_samples],
    }
    
    # Risk features (numeric)
    data['risk_score'] = np.random.beta(2, 5, n_samples)
    data['payment_score'] = np.random.beta(3, 2, n_samples)
    data['debt_ratio'] = np.random.beta(2, 3, n_samples)
    data['income_level'] = np.random.lognormal(10, 1.5, n_samples)
    data['credit_history_months'] = np.random.gamma(3, 10, n_samples)
    data['num_credit_lines'] = np.random.poisson(3, n_samples)
    data['utilization_rate'] = np.random.beta(3, 2, n_samples)
    data['num_inquiries'] = np.random.poisson(2, n_samples)
    
    # Categorical features
    data['employment_type'] = np.random.choice(
        ['Full-time', 'Part-time', 'Self-employed', 'Unemployed'], 
        n_samples, 
        p=[0.6, 0.2, 0.15, 0.05]
    )
    data['region'] = np.random.choice(['North', 'South', 'East', 'West'], n_samples)
    data['product_type'] = np.random.choice(['A', 'B', 'C'], n_samples, p=[0.5, 0.3, 0.2])
    
    # Create target
    risk_factor = (
        3.0 * data['risk_score'] + 
        2.5 * data['payment_score'] + 
        2.0 * data['debt_ratio'] + 
        1.5 * data['utilization_rate'] +
        0.5 * (data['num_inquiries'] / 10) +
        -0.3 * np.log1p(data['income_level'] / 10000) +
        -0.2 * np.log1p(data['credit_history_months'] / 12) +
        np.random.normal(0, 0.5, n_samples)
    )
    
    default_prob = 1 / (1 + np.exp(-2 * (risk_factor - np.median(risk_factor))))
    data['target'] = np.random.binomial(1, default_prob)
    
    # Adjust default rate
    if data['target'].mean() > 0.25:
        threshold = np.percentile(default_prob, 75)
        data['target'] = (default_prob > threshold).astype(int)
    elif data['target'].mean() < 0.10:
        threshold = np.percentile(default_prob, 90)
        data['target'] = (default_prob > threshold).astype(int)
    
    # Add missing values  
    missing_cols = ['income_level', 'credit_history_months']
    for col in missing_cols:
        missing_idx = np.random.choice(n_samples, size=int(0.05 * n_samples), replace=False)
        data[col][missing_idx] = np.nan
    
    df = pd.DataFrame(data)
    
    print(f"Dataset created:")
    print(f"  Shape: {df.shape}")
    print(f"  Default rate: {df['target'].mean():.2%}")
    print(f"  Date range: {df['app_dt'].min().date()} to {df['app_dt'].max().date()}")
    print(f"  Missing values: {df.isnull().sum().sum()}")
    
    return df

# Generate data with fixed seed
try:
    df = create_sample_data(n_samples=10000, seed=42)
    print("\n✓ Data generated successfully!")
    display(df.head())
except Exception as e:
    print(f"✗ Error generating data: {e}")

Dataset created:
  Shape: (10000, 14)
  Default rate: 25.00%
  Date range: 2022-01-01 to 2023-02-21
  Missing values: 1000

✓ Data generated successfully!


,app_id,app_dt,risk_score,payment_score,debt_ratio,income_level,credit_history_months,num_credit_lines,utilization_rate,num_inquiries,employment_type,region,product_type,target
0,1,2022-01-01 00:00:00,0.353677,0.576435,0.604789,66096.391145,7.454650,5,0.720952,3,Full-time,East,A,0
1,2,2022-01-01 01:00:00,0.248558,0.935896,0.245044,13009.413385,26.725380,3,0.319858,3,Part-time,North,A,0
2,3,2022-01-01 02:00:00,0.415959,0.924080,0.405689,65966.324008,27.662485,5,0.657510,1,Self-employed,West,A,1
3,4,2022-01-01 03:00:00,0.159968,0.626639,0.602617,46467.378787,44.511552,3,0.702942,0,Full-time,East,A,0
4,5,2022-01-01 04:00:00,0.550283,0.854347,0.577085,14137.477402,11.483100,3,0.282394,1,Unemployed,East,A,1


## 4. Configure Pipeline

In [14]:
# Create configuration
try:
    config = Config(
        # Core columns
        id_col='app_id',
        time_col='app_dt',
        target_col='target',
        
        # Enable DUAL PIPELINE
        enable_dual_pipeline=True,
        
        # Raw pipeline settings
        raw_imputation_strategy='median',
        raw_outlier_method='iqr',
        raw_outlier_threshold=1.5,
        
        # Data split
        use_test_split=True,
        test_size_row_frac=0.2,
        oot_window_months=2,
        
        # Feature engineering
        rare_threshold=0.01,
        psi_threshold=0.30,
        iv_min=0.01,
        rho_threshold=0.95,
        
        # Model settings - IMPORTANT: Use Config's existing attributes
        cv_folds=3,
        hpo_method='random',  # Use random search (faster than optuna)
        hpo_timeout_sec=30,
        hpo_trials=5,
        
        # Output
        output_folder='outputs_dual_example',
        output_excel_path='dual_pipeline_results.xlsx',
        
        random_state=42
    )
    
    print("✓ Configuration created successfully!")
    print(f"\nSettings:")
    print(f"  Dual Pipeline: {config.enable_dual_pipeline}")
    print(f"  HPO Method: {config.hpo_method}")
    print(f"  Raw Imputation: {config.raw_imputation_strategy}")
    print(f"  Raw Outlier Method: {config.raw_outlier_method}")
    print(f"  Output: {config.output_folder}")
    
except Exception as e:
    print(f"✗ Error creating configuration: {e}")

✓ Configuration created successfully!

Settings:
  Dual Pipeline: True
  HPO Method: random
  Raw Imputation: median
  Raw Outlier Method: iqr
  Output: outputs_dual_example


## 5. Run Pipeline

In [15]:
# Run pipeline with error handling
print("Preparing to run pipeline...")

try:
    # Set random seed before pipeline run for consistency
    import numpy as np
    import random
    np.random.seed(42)
    random.seed(42)
    
    # Create pipeline instance
    pipeline = RiskModelPipeline(config)
    print("✓ Pipeline instance created")
    
    # Run pipeline
    print("\n" + "="*60)
    print("STARTING DUAL PIPELINE EXECUTION")
    print("="*60 + "\n")
    
    start_time = time.time()
    pipeline.run(df)
    elapsed = time.time() - start_time
    
    print(f"\n✓ Pipeline completed in {elapsed:.2f} seconds")
    
except Exception as e:
    print(f"\n✗ Pipeline error: {e}")
    print("\nPossible solutions:")
    print("  1. Check if all required packages are installed")
    print("  2. Verify numpy/pandas compatibility")
    print("  3. Run: pip install git+https://github.com/selimoksuz/risk-model-pipeline.git")
    print("\nDetailed error:")
    import traceback
    traceback.print_exc()

# Note: Pipeline output will appear here when cell is run

Preparing to run pipeline...
✓ Pipeline instance created

STARTING DUAL PIPELINE EXECUTION

[15:22:22] >> 1) Veri yukleme & hazirlik basliyor | CPU=0% RAM=22%
   - Veri boyutu: 10,000 satir x 14 sutun
   - Target orani: 25.00%
   - Random seed: 42
[15:22:22] â--  1) Veri yukleme & hazirlik bitti (0.10s) — OK | CPU=1% RAM=22%
[15:22:22] >> 2) Giris dogrulama & sabitleme basliyor | CPU=0% RAM=22%
[15:22:22] â--  2) Giris dogrulama & sabitleme bitti (0.12s) — OK | CPU=0% RAM=22%
[15:22:22] >> 3) Degisken siniflamasi basliyor | CPU=1% RAM=22%
   - numeric=8, categorical=4
[15:22:22] â--  3) Degisken siniflamasi bitti (0.11s) — OK | CPU=0% RAM=22%
[15:22:22] >> 4) Eksik & Nadir deger politikasi basliyor | CPU=0% RAM=22%
[15:22:22] â--  4) Eksik & Nadir deger politikasi bitti (0.11s) — OK | CPU=2% RAM=22%
[15:22:23] >> 5) Zaman bolmesi (Train/Test/OOT) basliyor | CPU=0% RAM=22%
   - Train=6809, Test=1702, OOT=1489
[15:22:23] â--  5) Zaman bolmesi (Train/Test/OOT) bitti (0.12s) — OK | CPU=5% 

## 6. Review Results

In [7]:
# Review results with error handling
try:
    if hasattr(pipeline, 'models_summary_') and pipeline.models_summary_ is not None:
        print("="*60)
        print("MODEL PERFORMANCE SUMMARY")
        print("="*60)
        
        summary = pipeline.models_summary_
        
        # Check for Gini column
        gini_col = None
        for col in ['Gini_OOT', 'gini_oot', 'Gini_Test', 'gini_test']:
            if col in summary.columns:
                gini_col = col
                break
        
        if gini_col:
            print(f"\nTop 5 Models by {gini_col}:")
            top_models = summary.nlargest(5, gini_col)
            print(top_models[['model_name', gini_col]].to_string())
        else:
            print("\nModel Summary:")
            print(summary.head().to_string())
        
        # Check for pipeline comparison
        if 'pipeline' in summary.columns:
            print("\n" + "="*60)
            print("PIPELINE COMPARISON")
            print("="*60)
            
            for pipeline_type in ['WOE', 'RAW']:
                pipeline_models = summary[summary['pipeline'] == pipeline_type]
                if not pipeline_models.empty:
                    print(f"\n{pipeline_type} Pipeline:")
                    print(f"  Models: {len(pipeline_models)}")
                    if gini_col and gini_col in pipeline_models.columns:
                        print(f"  Best {gini_col}: {pipeline_models[gini_col].max():.4f}")
                        print(f"  Mean {gini_col}: {pipeline_models[gini_col].mean():.4f}")
    else:
        print("No model summary available.")
        
except Exception as e:
    print(f"Error reviewing results: {e}")

MODEL PERFORMANCE SUMMARY

Top 5 Models by Gini_OOT:
        model_name  Gini_OOT
6     RAW_Logit_L2  0.867930
11         RAW_GAM  0.866668
8   RAW_ExtraTrees  0.863510
10    RAW_LightGBM  0.861277
9      RAW_XGBoost  0.860457

PIPELINE COMPARISON

WOE Pipeline:
  Models: 6
  Best Gini_OOT: 0.0000
  Mean Gini_OOT: 0.0000

RAW Pipeline:
  Models: 6
  Best Gini_OOT: 0.8679
  Mean Gini_OOT: 0.8612


## 7. Export Reports

In [8]:
# Export reports
try:
    pipeline.export_reports()
    print("✓ Reports exported successfully!")
    
    # List generated files
    import os
    if os.path.exists(config.output_folder):
        files = os.listdir(config.output_folder)
        print(f"\nGenerated {len(files)} files in '{config.output_folder}':")
        for f in sorted(files)[:10]:
            size = os.path.getsize(os.path.join(config.output_folder, f)) / 1024
            print(f"  - {f} ({size:.1f} KB)")
        if len(files) > 10:
            print(f"  ... and {len(files)-10} more files")
            
except Exception as e:
    print(f"Error exporting reports: {e}")

✓ Reports exported successfully!

Generated 4 files in 'outputs_dual_example':
  - best_model_20250904_140258_44dce952.joblib (1.2 KB)
  - dual_pipeline_results.xlsx (24.9 KB)
  - final_vars_20250904_140258_44dce952.json (0.1 KB)
  - woe_mapping_20250904_140258_44dce952.json (26.1 KB)


## Troubleshooting

If you encounter any errors:

1. **numpy.dtype size changed error**:
   ```bash
   pip install git+https://github.com/selimoksuz/risk-model-pipeline.git
   ```

2. **Import errors**:
   ```bash
   pip install --force-reinstall git+https://github.com/selimoksuz/risk-model-pipeline.git
   ```

3. **Memory issues**:
   - Reduce n_samples in create_sample_data()
   - Reduce hpo_trials in config

4. **Create fresh environment**:
   ```bash
   python -m venv fresh_env
   fresh_env\Scripts\activate  # Windows
   pip install git+https://github.com/selimoksuz/risk-model-pipeline.git
   ```